# Data

In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("Expss4/EXP-Last-Place")
dataset

In [ ]:
dataset.set_format(type='torch')

In [ ]:
id2label = {110:0, 120:1, 150:2}

In [ ]:
def mapmap(row):
    return {'data': row['data'].unsqueeze(0).unsqueeze(0), 'label': id2label(int(row['label']))}

dataset['train'] = dataset['train'].map()
dataset

In [ ]:
dataset['train']['label'][0]

In [ ]:
dataset = dataset['train'].train_test_split(0.2)
dataset

In [ ]:
torch.tensor(dataset['train']['data'][0]).shape

In [ ]:
# import torch
# arrs = torch.tensor(dataset['train']['arr']).reshape(5556, 8, 865)
# arrs.shape

In [ ]:
# labels = torch.tensor(dataset['train']['label']).reshape(5556, 8, 1)
# labels.shape

In [ ]:
# labels = [label for label in labels]
# len(labels)

In [ ]:
# from datasets import Dataset
# new_dataset = Dataset.from_dict({'arr': arrs, 'label': labels})
# new_dataset

In [ ]:
# new_dataset.push_to_hub('Expss4/train_seq_pre_gj_8in1')

In [ ]:
# import numpy as np
# np.array(dataset['train']['processed'][0]).shape

In [ ]:
# from datasets import load_dataset

# # If the dataset is gated/private, make sure you have run huggingface-cli login
# dataset = load_dataset("Expss4/train_seq_pre_gj_8in1")

In [ ]:
# dataset = dataset['train'].train_test_split(0.2)
# dataset

In [ ]:
# dataset['train']['label'][0]

In [ ]:
# def go_out(example):
#     return {'label':example['label'][0][0]}
# dataset = dataset.map(go_out)

In [ ]:
# dataset.push_to_hub('Expss4/train_seq_pre_gj_8in1')

# DataLoader

In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class Loader(Dataset):
#     def __init__(self, datadict=None):
#         self.datadict = datadict
#         self.datadict.set_format(type='torch')

#     def __len__(self):
#         return len(self.datadict)

#     def __getitem__(self, idx):
#         data = self.datadict['data'][idx].unsqueeze(0).unsqueeze(0)
#         label = id2label[int(self.datadict['label'][idx])]
#         return {'data':data, 'label':label}

In [ ]:
# train_loader = DataLoader(Loader(dataset['train']), batch_size=2)
# val_loader = DataLoader(Loader(dataset['test']), batch_size=2)

# Model 103

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding),
            nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding),
            nn.GELU(),
            nn.BatchNorm2d(in_channels),
            nn.Dropout2d(0.2),
            nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding),
            nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding),
            nn.GELU(),
            nn.BatchNorm2d(in_channels),
            nn.Dropout2d(0.2),
            nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding),
            nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding),
            nn.SiLU(),
        )

        self.out = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
        )
    
    def forward(self, x):
        y = self.cnn(x)
        y += x
        y = self.out(y)

        return y


In [ ]:
class RealResnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.res_1 = ResidualBlock(1, 3, 3, 1)
        self.res_2 = ResidualBlock(3, 3, 5, 2)
        self.res_3 = ResidualBlock(3, 8, 5, 2)
        self.res_4 = ResidualBlock(8, 8, 5, 2)
        self.maxpool2d_5 = nn.MaxPool2d(2)
        
        # self.res_6 = ResidualBlock(8, 16, 3, 1)
        # self.res_7 = ResidualBlock(16, 16, 5, 2)
        # self.res_8 = ResidualBlock(16, 32, 5, 2)
        # self.res_9 = ResidualBlock(32, 32, 5, 2)
        # self.maxpool2d_10 = nn.MaxPool2d(2)
        
        # self.res_11 = ResidualBlock(32, 64, (1, 7), (0, 3))
        # self.res_12 = ResidualBlock(64, 64, (1, 7), (0, 3))
        # self.res_13 = ResidualBlock(64, 128, (1, 9), (0, 4))
        # self.res_14 = ResidualBlock(128, 128, (1, 9), (0, 4))
        # self.maxpool2d_15 = nn.MaxPool2d(2)

        self.classify_16 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(288, 128),
            nn.ReLU(),
            # nn.Linear(2048, 512),
            nn.Dropout(0.2),
            # nn.ReLU(),
            nn.Linear(128, 3),
            nn.Softmax(),
        )
  
    
    def forward(self, x):
        x = self.res_1(x)
        x = self.res_2(x)
        x = self.res_3(x)
        x = self.res_4(x)
        x = self.maxpool2d_5(x)
        
        # x = self.res_6(x)
        # x = self.res_7(x)
        # x = self.res_8(x)
        # x = self.res_9(x)
        # x = self.maxpool2d_10(x)
        
        # x = self.res_11(x)
        # x = self.res_12(x)
        # x = self.res_13(x)
        # x = self.res_14(x)
        # x = self.maxpool2d_15(x)

        x = self.classify_16(x)

        return x

# model = RealResnet()

In [ ]:
import torch
model = torch.load('tst_model103.pt') # best model
model

In [ ]:
import torchsummary

torchsummary.summary(model)

In [ ]:
input = torch.empty(8, 18).unsqueeze(0).unsqueeze(0)
print(input.size())
model(input).size()

In [ ]:
model = torch.load('tst_model103.pt')

# Model

In [ ]:
class c1dT(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(8, 32, 1),
            nn.Conv1d(32, 32, 3),
            nn.SiLU(),
            nn.Dropout2d(),
            nn.Conv1d(32, 64, 3),
            nn.Conv1d(64, 128, 3),
            nn.SiLU(),
        )
        self.flatten = nn.Flatten(0, -1)
        self.dense = nn.Sequential(
            nn.Linear(1536, 1024),
            nn.SiLU(),
            nn.Linear(1024, 512),
            nn.Dropout(),
            nn.SiLU(),
            nn.Linear(512, 128),
            nn.SiLU(),
            nn.Linear(128, 3),
            nn.Softmax(),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

model = c1dT()
input = torch.empty([8, 18])
print(input.size())
model(input).size()

# Model 2

In [ ]:
class smallres(nn.Module):
    def __init__(self):
        super().__init__()
        self.res_1 = ResidualBlock(1, 3, 3, 1)
        self.res_2 = ResidualBlock(3, 3, 5, 2)

        self.flatten = nn.Flatten(0, -1)

        self.dense = nn.Sequential(
            nn.LayerNorm(432),
            nn.Linear(432, 512),
            nn.SiLU(),
            nn.Linear(512, 432),
            nn.Dropout(),
            nn.SELU(),
            nn.Linear(432, 128),
            nn.SELU(),
            nn.Linear(128, 3),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.res_1(x)
        x = self.res_2(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

# model = smallres()
# input = torch.empty([1, 1, 8, 18])
# print(input.size())
# model(input).size()

In [ ]:
model = torch.load('smallres_model_a134.pt')
model

In [ ]:
input_2 = torch.empty([8, 18]).to(device)
input.size(), input_2.size()

In [ ]:
input.size()

model(input_2)

# Train

In [ ]:
model = torch.load('tst_model99.pt')

In [ ]:
# for Model 103
from tqdm import tqdm
from torch.optim import AdamW
from sklearn.metrics import f1_score 
from torch.optim.lr_scheduler import CosineAnnealingLR

torch.backends.cudnn.benchmark =  True
torch.backends.cudnn.enabled =  True

device = 'cuda' if torch.cuda.is_available else 'cpu'

early_stop_patients = 22
patients_count = 0

# Fine-tuning parameters
num_epochs = 150
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=150)
loss_fn = nn.CrossEntropyLoss()

model.to(device)
# scaler = torch.cuda.amp.GradScaler()

total_train_loss = []
total_test_loss = []
total_f1 = []

maxf1 = 0
# Fine-tuning loop
for epoch in range(num_epochs):
    # Training
    model.train()
    train_avg_loss = 0
    ty_true = []
    ty_pred = []
    for idx, row in tqdm(enumerate(dataset['train']), total=len(dataset['train'])):
      torch.cuda.empty_cache()

      input = torch.tensor(row['data']).unsqueeze(0).unsqueeze(0).to(device)
      label = id2label[int(row['label'])]

      outputs = model(input).squeeze()
      loss = loss_fn(outputs, F.one_hot(torch.tensor(label), 3).float().to(device))
      # scaler.scale(loss).backward()
      loss.backward()

      ty_true.append(label)
      ty_pred.append(torch.argmax(outputs.to('cpu'), -1))

      if (idx % 128) == 0:
        # scaler.step(optimizer)
        optimizer.step()
        optimizer.zero_grad()
        # scaler.update()
      train_avg_loss += loss.item()
    train_avg_loss /= len(dataset['train'])
    total_train_loss.append(train_avg_loss)

    # Validation
    model.eval()
    test_avg_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
      for idx, row in tqdm(enumerate(dataset['test']), total=len(dataset['test'])):

        input = torch.tensor(row['data']).unsqueeze(0).unsqueeze(0).to(device)
        label = id2label[int(row['label'])]

        val_outputs = model(input).squeeze()
        val_loss = loss_fn(val_outputs, F.one_hot(torch.tensor(label), 3).float().to(device))

        y_true.append(label)
        y_pred.append(torch.argmax(val_outputs.to('cpu'), -1))
        
        test_avg_loss += val_loss.item()
          
      test_avg_loss /= len(dataset['test'])
      total_test_loss.append(test_avg_loss)

      f1score = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
      total_f1.append(f1score)

      print(f"Epoch {epoch+1}/{num_epochs}: Avg Loss: {train_avg_loss}, Avg Val Loss: {test_avg_loss}, \ntF1_score: {f1_score(y_true=ty_true, y_pred=ty_pred, average='macro')}, vF1_score: {f1score}")
      
      # early stop
      
      torch.save(model, f'tst_model{epoch+100}.pt')

In [ ]:
# for Model
from tqdm import tqdm
from torch.optim import AdamW
from sklearn.metrics import f1_score 
from torch.optim.lr_scheduler import CosineAnnealingLR

torch.backends.cudnn.benchmark =  True
torch.backends.cudnn.enabled =  True

device = 'cuda' if torch.cuda.is_available else 'cpu'

early_stop_patients = 22
patients_count = 0

# Fine-tuning parameters
num_epochs = 150
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=150)
loss_fn = nn.CrossEntropyLoss()

model.to(device)
# scaler = torch.cuda.amp.GradScaler()

total_train_loss = []
total_test_loss = []
total_f1 = []

maxf1 = 0
# Fine-tuning loop
for epoch in range(num_epochs):
    # Training
    model.train()
    train_avg_loss = 0
    ty_true = []
    ty_pred = []
    for idx, row in tqdm(enumerate(dataset['train']), total=len(dataset['train'])):
      torch.cuda.empty_cache()

      input = torch.tensor(row['data']).to(device)
      label = id2label[int(row['label'])]
      outputs = model(input)
      loss = loss_fn(outputs, F.one_hot(torch.tensor(label), 3).float().to(device))
      # scaler.scale(loss).backward()
      loss.backward()

      ty_true.append(label)
      ty_pred.append(torch.argmax(outputs.to('cpu'), -1))

      if (idx % 128) == 0:
        # scaler.step(optimizer)
        optimizer.step()
        optimizer.zero_grad()
        # scaler.update()
      train_avg_loss += loss.item()
    train_avg_loss /= len(dataset['train'])
    total_train_loss.append(train_avg_loss)

    # Validation
    model.eval()
    test_avg_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
      for idx, row in tqdm(enumerate(dataset['test']), total=len(dataset['test'])):

        input = torch.tensor(row['data']).to(device)
        label = id2label[int(row['label'])]

        val_outputs = model(input)
        val_loss = loss_fn(val_outputs, F.one_hot(torch.tensor(label), 3).float().to(device))

        y_true.append(label)
        y_pred.append(torch.argmax(val_outputs.to('cpu'), -1))
        
        test_avg_loss += val_loss.item()
          
      test_avg_loss /= len(dataset['test'])
      total_test_loss.append(test_avg_loss)

      f1score = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
      total_f1.append(f1score)

      print(f"Epoch {epoch+1}/{num_epochs}: Avg Loss: {train_avg_loss}, Avg Val Loss: {test_avg_loss}, \ntF1_score: {f1_score(y_true=ty_true, y_pred=ty_pred, average='macro')}, vF1_score: {f1score}")
      
      # early stop
      
      torch.save(model, f'tst_model_a{epoch}.pt')

In [ ]:
# for Model 2
from tqdm import tqdm
from torch.optim import AdamW
from sklearn.metrics import f1_score 
from torch.optim.lr_scheduler import CosineAnnealingLR

torch.backends.cudnn.benchmark =  True
torch.backends.cudnn.enabled =  True

device = 'cuda' if torch.cuda.is_available else 'cpu'

early_stop_patients = 22
patients_count = 0

# Fine-tuning parameters
num_epochs = 150
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=150)
loss_fn = nn.CrossEntropyLoss()

model.to(device)
# scaler = torch.cuda.amp.GradScaler()

total_train_loss = []
total_test_loss = []
total_f1 = []

maxf1 = 0
# Fine-tuning loop
for epoch in range(num_epochs):
    # Training
    model.train()
    train_avg_loss = 0
    ty_true = []
    ty_pred = []
    for idx, row in tqdm(enumerate(dataset['train']), total=len(dataset['train'])):
      torch.cuda.empty_cache()

      input = torch.tensor(row['data']).unsqueeze(0).unsqueeze(0).to(device)
      label = id2label[int(row['label'])]
      outputs = model(input)
      loss = loss_fn(outputs, F.one_hot(torch.tensor(label), 3).float().to(device))
      # scaler.scale(loss).backward()
      loss.backward()

      ty_true.append(label)
      ty_pred.append(torch.argmax(outputs.to('cpu'), -1))

      if (idx % 128) == 0:
        # scaler.step(optimizer)
        optimizer.step()
        optimizer.zero_grad()
        # scaler.update()
      train_avg_loss += loss.item()
    train_avg_loss /= len(dataset['train'])
    total_train_loss.append(train_avg_loss)

    # Validation
    model.eval()
    test_avg_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
      for idx, row in tqdm(enumerate(dataset['test']), total=len(dataset['test'])):

        input = torch.tensor(row['data']).unsqueeze(0).unsqueeze(0).to(device)
        label = id2label[int(row['label'])]

        val_outputs = model(input)
        val_loss = loss_fn(val_outputs, F.one_hot(torch.tensor(label), 3).float().to(device))

        y_true.append(label)
        y_pred.append(torch.argmax(val_outputs.to('cpu'), -1))
        
        test_avg_loss += val_loss.item()
          
      test_avg_loss /= len(dataset['test'])
      total_test_loss.append(test_avg_loss)

      f1score = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
      total_f1.append(f1score)

      print(f"Epoch {epoch+1}/{num_epochs}: Avg Loss: {train_avg_loss}, Avg Val Loss: {test_avg_loss}, \ntF1_score: {f1_score(y_true=ty_true, y_pred=ty_pred, average='macro')}, vF1_score: {f1score}")
      
      # early stop
      
      torch.save(model, f'tst_model_a{epoch}.pt')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(total_train_loss)
plt.plot(total_test_loss)
plt.plot(total_f1)
plt.show()

In [ ]:
# model = torch.load('test_model_a134.pt') # this model is better in public
model = torch.load('tst_model103.pt') # but this model in private

In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("Expss4/EXP-Last-Place-Test")
dataset

In [ ]:
id2label = {110:0, 120:1, 150:2}
label2id = {0:110, 1:120, 2:150}

In [ ]:
y_pred = []
with torch.no_grad():
      for idx, row in tqdm(enumerate(dataset['train']), total=len(dataset['train'])):

        input = torch.tensor(row['data']).unsqueeze(0).unsqueeze(0).to(device)

        val_outputs = model(input)
        y_pred.append(label2id[int(torch.argmax(val_outputs.to('cpu'), -1))])

In [ ]:
y_pred

In [ ]:
import pandas as pd

sub = pd.read_csv('sample_submission.csv')
sub

In [ ]:
id_test = [example['id'] for example in dataset['train']]

In [ ]:
p = id_test
p

In [ ]:
df = pd.DataFrame({
    'predict': y_pred,
    'id': p
})
df

In [ ]:
result = pd.merge(sub, df[['id', 'predict']], on='id', how='left')
result

In [ ]:
result = result.drop('predict_x', axis = 1)

In [ ]:
result.rename(columns={'predict_y': 'predict'}, inplace=True)

In [ ]:
result

In [ ]:
result.loc[0:2, 'predict'] = [110, 150, 150]
result

In [ ]:
result.to_csv('mixueicecreamcha103.csv',index= False)